In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

customers_path = os.path.join(os.getcwd(), 'Customers.csv')
products_path = os.path.join(os.getcwd(), 'Products.csv')
transactions_path = os.path.join(os.getcwd(), 'Transactions.csv')

customers = pd.read_csv(customers_path, dtype={'CustomerID': 'string', 'Region': 'string', 'CustomerName': 'string'})
products = pd.read_csv(products_path, dtype={'ProductID': 'string', 'Category': 'string', 'ProductName': 'string', 'Price': 'float'})
transactions = pd.read_csv(transactions_path, dtype={'TransactionID': 'string', 'CustomerID': 'string', 'ProductID': 'string', 'Quantity': 'int', 'TotalValue': 'float'}, parse_dates=['TransactionDate'])

merged_data = transactions.merge(customers, on="CustomerID", how="inner").merge(products, on="ProductID", how="inner")

customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],
    'Quantity': 'sum',
    'ProductID': 'nunique',
    'TransactionID': 'count',
    'Region': 'first',
}).reset_index()
customer_features.columns = ['CustomerID', 'TotalSpending', 'AvgSpending', 'TotalQuantity', 'UniqueProducts', 'PurchaseFrequency', 'Region']

customer_features = pd.get_dummies(customer_features, columns=['Region'], drop_first=True)

scaler = StandardScaler()
numeric_features = ['TotalSpending', 'AvgSpending', 'TotalQuantity', 'UniqueProducts', 'PurchaseFrequency']
customer_features[numeric_features] = scaler.fit_transform(customer_features[numeric_features])

inertia = []
db_indices = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(customer_features[numeric_features])
    inertia.append(kmeans.inertia_)
    db_indices.append(davies_bouldin_score(customer_features[numeric_features], kmeans.labels_))

plt.figure(figsize=(10, 6))
plt.plot(range(2, 10), inertia, marker='o')
plt.title("Elbow Method for Optimal Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(2, 10), db_indices, marker='o', color='red')
plt.title("Davies-Bouldin Index for Optimal Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("DB Index")
plt.show()

optimal_k = db_indices.index(min(db_indices)) + 2
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
customer_features['Cluster'] = kmeans.fit_predict(customer_features[numeric_features])

db_index = davies_bouldin_score(customer_features[numeric_features], customer_features['Cluster'])
print(f"Optimal Number of Clusters: {optimal_k}")
print(f"Davies-Bouldin Index: {db_index:.2f}")

plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=customer_features['TotalSpending'],
    y=customer_features['AvgSpending'],
    hue=customer_features['Cluster'],
    palette='viridis',
    s=100
)
plt.title("Customer Segments (Clusters)")
plt.xlabel("Total Spending (Standardized)")
plt.ylabel("Average Spending (Standardized)")
plt.legend(title="Cluster")
plt.show()

customer_features.to_csv('Customer_Segments_Optimized.csv', index=False)
print("Customer segmentation results saved to 'Customer_Segments_Optimized.csv'")
